In [2]:
import netCDF4 as nc
from netCDF4 import Dataset
import os
import numpy as np
import pandas as pd
from datetime import datetime, timezone, timedelta
from tqdm import tqdm

from scipy.spatial import cKDTree

import sys, os
sys.path.append('../src')
import Biologging_Toolkit.utils.rain_empirical_utils as R_Utils

c:\Users\barsro\Documents\SES_tags\notebooks\../src\Biologging_Toolkit


In [3]:
depids = ['ml17_280a', 'ml18_296a', 'ml18_294b', 'ml19_292a', 'ml19_292b', 'ml19_293a', 'ml19_294a', 'ml20_293a',
          'ml20_296b', 'ml20_313a', 'ml21_295a', 'ml21_305b']
gpm_path = "E:/individus_filtered/"
path = "E:/individus_filtered/"
Anatole_path = "F:/"

In [10]:
time_off = 5*60
lat_off = 0.2
lon_off = 0.2

In [18]:
for depid in depids : 
    df_Anatole = pd.read_csv(os.path.join(Anatole_path, f'{depid}_dive.csv'))
    df_Bob1 = pd.read_csv(os.path.join(path, depid, f'{depid}_dive.csv'))
    df_Bob1[["end_lat", "end_lon"]] = df_Anatole[["end_lat", "end_lon"]]
    df_Bob1.to_csv(os.path.join(path, depid, f'{depid}_dive.csv'), index = None)

In [17]:
df_Bob1.head()

,dive,begin_time,end_time,zn1,zn2,zn1_up,zn2_up,zn1_down,zn2_down,threhsold_up,...,lstm_cfosat,cfosat_wind,two_regime,end_lat,end_lon,acc_drifts,cluster_drifts,flash,cube_gpm,precipitation_GPM
0,0.0,1.507334e+09,1.507335e+09,NaN,NaN,NaN,NaN,NaN,NaN,157.202545,...,14.646183,NaN,14.048231,-47.306847,67.766945,0,1,2.0,NaN,NaN
1,1.0,1.507335e+09,1.507337e+09,NaN,NaN,NaN,NaN,NaN,NaN,134.639832,...,13.850940,NaN,13.235098,-47.301159,67.748039,0,1,1.0,NaN,0.014598
2,2.0,1.507337e+09,1.507338e+09,NaN,NaN,NaN,NaN,NaN,NaN,125.335068,...,13.279770,NaN,13.010594,-47.297382,67.728798,0,1,1.0,0.020625,0.020260
3,3.0,1.507338e+09,1.507339e+09,NaN,NaN,NaN,NaN,NaN,NaN,124.460243,...,13.404723,NaN,12.863159,-47.292889,67.709740,0,1,1.0,NaN,0.025509
4,4.0,1.507339e+09,1.507341e+09,NaN,NaN,NaN,NaN,NaN,NaN,133.336792,...,12.466095,NaN,12.163302,-47.289333,67.688240,1,1,0.0,NaN,0.043237


---
# Cylinder interpolation
---

In [ ]:
for depid in depids[0:1] :
    print(f"Working on {depid}")
    path = 'E:/individus_filtered/'
    X = 0.036

    nc_gpm = Dataset(os.path.join(path, depid, f'{depid}_gpm.nc'))

    lats = nc_gpm['lat'][:]
    lons = nc_gpm['lon'][:]
    times = nc_gpm['time']
    precip = nc_gpm['precipitation'][:]  # Shape: [time, lat, lon]

    base_time = datetime(1980, 1, 6, 0, 0, 0)  # sans tzinfo
    gpm_dates = np.array([
        base_time + timedelta(seconds=float(sec))
        for sec in nc_gpm['time'][:]
    ])

    lon_grid, lat_grid = np.meshgrid(lons, lats)
    points = np.column_stack((lat_grid.ravel(), lon_grid.ravel()))
    tree = cKDTree(points)

    df = pd.read_csv(os.path.join(path, depid, f'{depid}_dive_clean.csv'))
    
    def get_precip_at_point(row):
        lat, lon, t = row['lat'], row['lon'], pd.to_datetime(row["end_time"]).to_pydatetime()
        if np.isnan(lat) or np.isnan(lon):
            return np.nan
        dist, idx = tree.query([lat, lon], k=len(points), distance_upper_bound=X)
        valid_idx = idx[np.isfinite(dist)]
        if not len(valid_idx):
            return np.nan
        time_diff = np.abs(gpm_dates - t)
        time_idx = time_diff.argmin()
        gpm_values = precip[time_idx].ravel()[valid_idx]
        return np.nansum(gpm_values)

    df[f'cylinder_gpm_{X}'] = df.apply(get_precip_at_point, axis=1)
    # df.to_csv(os.path.join(path, depid, f'{depid}_dive.csv'), index = None)

Working on ml17_280a


C:\Users\barsro\AppData\Local\Temp\ipykernel_14556\868242781.py:26: UserWarning: Discarding nonzero nanoseconds in conversion.
  lat, lon, t = row['lat'], row['lon'], pd.to_datetime(row["end_time"]).to_pydatetime()


In [14]:
df.head()

,dive,begin_time,end_time,zn1,zn2,zn1_up,zn2_up,zn1_down,zn2_down,threhsold_up,...,mld_gradient,pensieri,hildebrand,temp10m,mld,cfosat_wind,lstm_cfosat,begin_wind,precipitation_GPM,cylinder_gpm_0.036
0,0.0,1.507334e+09,1.507335e+09,NaN,NaN,NaN,NaN,NaN,NaN,157.202545,...,NaN,14.880613,13.822303,4.177374,71.0,NaN,15.581713,NaN,NaN,NaN
1,1.0,1.507335e+09,1.507337e+09,NaN,NaN,NaN,NaN,NaN,NaN,134.639832,...,NaN,13.712882,12.973303,4.165605,58.0,NaN,15.214185,15.187502,0.014598,NaN
2,2.0,1.507337e+09,1.507338e+09,NaN,NaN,NaN,NaN,NaN,NaN,125.335068,...,NaN,13.394056,12.741852,4.153938,54.0,NaN,14.473380,15.082936,0.020260,NaN
3,3.0,1.507338e+09,1.507339e+09,NaN,NaN,NaN,NaN,NaN,NaN,124.460243,...,NaN,13.185580,12.590525,4.146532,55.0,NaN,14.536754,14.976356,0.025509,NaN
4,4.0,1.507339e+09,1.507341e+09,NaN,NaN,NaN,NaN,NaN,NaN,133.336792,...,NaN,12.206330,11.879160,4.142927,55.0,NaN,13.652518,14.824885,0.043237,NaN


---
# Cubic interpolation
---

In [19]:
"""Computes average of variables values of all ERA5 mesh points within a cube.
All mesh points within a defined latitude distance and longitude distance are kept, and within a maximum time offset.
Parameter
---------
time_off: 'int' or 'float'
Maximum time offset allowed for mesh point to be taken into consideration, in seconds.
lat_off : 'int' or 'float'
Maximum latitude offset allowed for mesh point to be taken into consideration, in degrees.
lon_off : 'int' or 'float'
Maximum longitude offset allowed for mesh point to be taken into consideration, in degrees.
variables : 'list' or None
List of variables from ERA5 to join to hydrophone position. None if you want all variables to be joined to data.
Returns
-------
None. Results are saved in self.df as 'cube_{time_off}_{lat_off}_{lon_off}_{variable}'
"""
for depid in depids[1:] :
    df = pd.read_csv(os.path.join(path, depid, f'{depid}_dive.csv'))
    gpm = nc.Dataset(os.path.join(gpm_path,depid, f'{depid}_gpm.nc'))
    temp_df = df[["end_time", "end_lat", "end_lon"]]
    temp_variable = np.full([len(temp_df)], np.nan)
    base_time = datetime(1980, 1, 6, 0, 0, 0, tzinfo=timezone.utc)

    gpm_time = np.array([
    (base_time + timedelta(seconds=int(second))).timestamp() 
    for second in gpm['time'][:].data
    ])
    gpm_lat, gpm_lon = gpm['lat'][:], gpm['lon'][:]
    precipitation = gpm['precipitation'][:].data
    for i, row in tqdm(temp_df.iterrows()):
        if row.isnull().any():  # Skip iteration if there is a nan value
            continue
        time_index = np.where((abs(gpm_time - row.end_time) <= time_off))[0]  # Get timestamps satisfyin time offset
        if (len(time_index) == 0):  # Skip iteration if no weather source is within time offset for this timestep
            temp_variable[i] = np.nan
            continue
        lat_index = np.where((abs(gpm_lat - row.end_lat) <= lat_off))[0]  # Get latitude indexes satisfying lon offset
        if (len(lat_index) == 0):  # Skip iteration if no weather source is within lat offset for this timestep
            temp_variable[i] = np.nan
            continue
        lon_index = np.where((abs(gpm_lon - row.end_lon) <= lon_off))[0]  # Get longitude indexes satisfying lon offset
        if (len(lon_index) == 0):  # Skip iteration if no weather source is within lon offset for this timestep
            temp_variable[i] = np.nan
            continue
        temp_value = []
        for time_ind in time_index:
            for lat_ind in lat_index:
                for lon_ind in lon_index:
                    temp_value.append(
                        precipitation[time_ind, lat_ind, lon_ind])  # Saving values satisfying the conditions
        temp_variable[i] = np.nanmean(temp_value)
    variable = f"cube_gpm"
    df[variable] = temp_variable
    df.to_csv(os.path.join(path, depid, f'{depid}_dive.csv'), index = None)


3344it [00:00, 5021.94it/s]
2387it [00:00, 4640.91it/s]
2081it [00:00, 3655.48it/s]
3793it [00:00, 4087.96it/s]
3798it [00:00, 6731.26it/s]
3619it [00:00, 5170.11it/s]
168it [00:00, 4992.12it/s]
4925it [00:00, 5080.83it/s]
2559it [00:00, 5441.05it/s]
4130it [00:00, 5501.58it/s]
2231it [00:00, 4677.13it/s]


In [9]:
print(temp_variable)

[     nan      nan 0.020625 ...      nan      nan      nan]
